In [27]:
import cv2
import joblib
import os
import time
from os import walk
from skimage import feature
import imageio
import numpy as np
from matplotlib import pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

import pickle


In [3]:
  
def segm_6(img):
#     blur the img then turn it into gray scale
    blur = cv2.GaussianBlur(img, (5, 5), 0)
    gray = cv2.cvtColor(blur, cv2.COLOR_BGR2GRAY)
    return gray


def make_to_left(img):
  a = img.sum(axis=0)
  a = a / a.max()
  if(a[:int(len(a)//8)].sum() < a[int(len(a)*7//8):].sum()):
    # flip the image
    return cv2.flip(img, 1)
  return img

def preprocess(img):

  holesimg = segm_6(img)
  # holesimg = segm_4(img_rgb)
  # holesimg = segm_6(img_rgb)
  holesimg = make_to_left(holesimg)
  return holesimg



In [24]:


def ImageSegmentation():
    path_IS = r"./test_seg"
    if not os.path.exists(path_IS):
        os.makedirs(path_IS)
    lstFiles = []  # nombre de imagenes
    path = r"./dataset"

    for (path, _, archivos) in walk(path):
        # print(path)
        for arch in archivos:
            (nomArch, ext) = os.path.splitext(arch)
            if (ext == ".JPG"):
                label = path.split("\\")[-1]
                # print(label)
                lstFiles.append(nomArch + ext)
                direc = path + "/" + nomArch + ext
                name = label + "_" + nomArch + ext
                # print(path + "/" + nomArch + ext)
                image = cv2.imread(direc, 0)

                if image is not None:
                    image = cv2.resize(image, (461, 260))
                    # cv2.imshow("ss",image)
                    image = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                                  cv2.THRESH_BINARY, 11, 2)
                    image = cv2.GaussianBlur(image, (5, 5), 0)

                    # image = np.sum(image, axis= 1)
                    # print(image.shape())
                    # holesimg = preprocess(direc)
                    # plt.imshow(image,'gray')
                    # # plt.title(titles[i])
                    # plt.xticks([]),plt.yticks([])
                    # plt.show()
                    imageio.imwrite(os.path.join(path_IS, name), image)


ImageSegmentation()


In [34]:

labels, features =[], []
for imagePath in os.listdir("test_seg"):
    # extract the label from the image path
    # we can get the label from the image name as the label is the first char of the image name
    # for example, the label of image 1_1.jpg is 1
    labels.append((int)(imagePath.split(os.path.sep)[-1][0]))
    
    # load the image, convert it to grayscale, and detect
    # edges in it
    image = cv2.imread("test_seg/" + imagePath)
    
    # extract Histogram of Oriented Gradients from the
    # test image
    # display the original image
    # if the image exists, display it and count the number of images
    if image is not None:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # image = cv2.resize(image, (461, 260))
        # (H, hogImage) = feature.hog(image, orientations=9,  pixels_per_cell=(32, 32), cells_per_block=(2, 2), transform_sqrt=True, block_norm="L1", visualize=True, feature_vector=True)
        # H = H.reshape(-1, 1)
        # pca = PCA(0.8).fit(H)
        # components = pca.transform(H)
        features.append(np.sum(image, axis= 1))

        

        # store the label and features in the csv file
        #csv.writer(features_file).writerow([labels[-1], pca_features])


In [39]:
# len(labels)
len(features)

1941

In [40]:
model = SVC(kernel='rbf', gamma=0.1, C=10)
trainingData, testData, trainingLabels, testLabels = train_test_split(features, labels, test_size=0.2, random_state=42)

model.fit(trainingData, trainingLabels)

# evaluate the model and update the accuracies list
# acc = model.a
# accuracy = acc
# print("[INFO] accuracy: {:.2f}%".format(acc * 100), flush=True)
# # dump the classifier to file
# print("[INFO] dumping classifier to file...", flush=True)
f = open("test_clf.pkl", "wb") # wb = write binary
f.write(pickle.dumps(model))
f.close()




In [44]:
# direc = "Image-Segmentation1"
direc = "./im"
# model = joblib.load("./SVM_classifier.pkl")
model = joblib.load("./test_clf.pkl")
pca_transformer = joblib.load("./Feature-Extraction/pca.pkl")
labels, times = [], []

count_right = 0
# for imagePath in os.listdir(direc):
for i, image in enumerate(testData):

    # image = cv2.imread(direc + "/" + imagePath)
    # image = cv2.resize(image, (461, 260))

    if image is not None:
        start = time.time()
        # image = preprocess(image)

        # (H, hogImage) = feature.hog(image, orientations=9,  pixels_per_cell=(16, 16), cells_per_block=(
        #     2, 2), transform_sqrt=True, block_norm="L1", visualize=True, feature_vector=True)
        # pca_features = pca_transformer.transform(H.reshape(1, -1))

        # label = model.predict(pca_features)
        label = model.predict(image.reshape(1, -1))
        end = time.time() - start
        if label == testLabels[i]:
            count_right += 1
        labels.append(label)
        times.append(end)


# with open(r'./result_labels.txt', 'w') as fp:
#     fp.write("\n".join(str(item) for item in labels))


with open(r'result_labels.txt', 'w') as fp:
    for item in labels:
        # write each item on a new line
        fp.write("%s\n" % item[0])
with open(r'./result_times.txt', 'w') as fp:
    fp.write("\n".join(str(item) for item in times))
    
print(count_right/len(testLabels))


C:\Users\77\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator PCA from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


0.2159383033419023
